In [1]:
import numpy as np
import itertools
from pathlib import Path
%matplotlib inline
import matplotlib.pyplot as plt
import sys
sys.path.append("../code/")
from dataloader import get_train_test
import pandas as pd

In [8]:
fold_num = 0

In [5]:
# load best parameters for rnn-individual and rnn-tree
best_rnn_individual = np.load("../code/baseline/result/rnn-individual-param-1.npy").item()
best_rnn_tree = np.load("../code/baseline/result/rnn-tree-param-1.npy").item()

In [7]:
# cnn-individual&cnn-tree best parameters
best_cnn_individual = np.load("../code/baseline/result/cnn-individual-param-1.npy").item()

In [132]:
# get prediction for rnn individual
rnn_individual_pred = {}
rnn_individual_error = {}
for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
    cell_type = best_rnn_individual[appliance][fold_num]['cell_type']
    hidden_size = best_rnn_individual[appliance][fold_num]['hidden_size']
    num_layers = best_rnn_individual[appliance][fold_num]['num_layers']
    lr = best_rnn_individual[appliance][fold_num]['lr']
    iters = best_rnn_individual[appliance][fold_num]['iters']
    bidirectional = best_rnn_individual[appliance][fold_num]['bidirectional']
    
    directory = "../code/baseline/rnn-tree/1/{}/{}/{}/{}/{}/{}/3000/0.0/".format(fold_num, cell_type,
                                                                                                hidden_size,
                                                                                                num_layers,
                                                                                                bidirectional,
                                                                                                lr)
    filename = "test-pred-{}-1-{}-{}-{}-{}-{}-3000-0.0-[\'{}\'].npy".format(fold_num, cell_type, hidden_size, num_layers,
                                                                             bidirectional, lr, appliance)
    
    full_path = directory + filename
    rnn_individual_pred[appliance] = np.asarray(np.load(full_path).item()[iters]).reshape(-1, 1, 112, 24)
    filename = "test-error-{}-1-{}-{}-{}-{}-{}-3000-0.0-[\'{}\'].npy".format(fold_num, cell_type, hidden_size, num_layers,
                                                                             bidirectional, lr, appliance)
    full_path = directory + filename
    rnn_individual_error[appliance] = np.load(full_path).item()[iters][appliance]
    

In [148]:
# get prediction for rnn tree
rnn_tree_pred = {}
rnn_tree_error = {}

cell_type = best_rnn_tree[fold_num]['cell_type']
hidden_size = best_rnn_tree[fold_num]['hidden_size']
num_layers = best_rnn_tree[fold_num]['num_layers']
lr = best_rnn_tree[fold_num]['lr']
iters = best_rnn_tree[fold_num]['iters']
bidirectional = best_rnn_tree[fold_num]['bidirectional']
order = best_rnn_tree[fold_num]['order']
directory = "../code/baseline/rnn-tree/1/{}/{}/{}/{}/{}/{}/3000/0.0/".format(fold_num, cell_type,
                                                                                            hidden_size,
                                                                                            num_layers,
                                                                                            bidirectional,
                                                                                            float(lr))
filename = "test-pred-{}-1-{}-{}-{}-{}-{}-3000-0.0-[\'{}\'].npy".format(fold_num, cell_type, hidden_size, num_layers,
                                                                         bidirectional, float(lr), str(order)[2:-2])

full_path = directory + filename
rnn_pred = np.asarray(np.load(full_path).item()[iters])
filename = "test-error-{}-1-{}-{}-{}-{}-{}-3000-0.0-[\'{}\'].npy".format(fold_num, cell_type, hidden_size, num_layers,
                                                                         bidirectional, float(lr), str(order)[2:-2])
full_path = directory + filename
rnn_tree_error= np.load(full_path).item()[iters] 

for idx, appliance in enumerate(order):
    print(idx, appliance)
    rnn_tree_pred[appliance] = rnn_pred[idx].reshape(-1, 1, 112,24)

0 fridge
1 hvac
2 dw
3 dr
4 mw


In [150]:
# get prediction for cnn individual
cnn_individual_pred = {}
cnn_individual_error = {}
for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
    iters= best_cnn_individual[appliance][fold_num]['iters']
    lr = best_cnn_individual[appliance][fold_num]['lr']
    
    directory = "../code/baseline/cnn-individual/1/{}/{}/{}/".format(fold_num, lr, iters)
    filename = "test-pred-{}.npy".format(appliance)
    full_path = directory + filename
    cnn_individual_pred[appliance] = np.load(full_path)
    
    filename = "test-loss-{}.npy".format(appliance)
    full_path = directory + filename
    cnn_individual_error[appliance] = np.load(full_path).item().cpu().data.numpy()[0]
    

In [151]:
# get prediction for cnn tree
cnn_pred = np.load("../code/baseline/cnn-tree/1/0/0.01/20000/0.0/test-pred-['dw', 'dr', 'hvac', 'fridge', 'mw'].npy").item()
cnn_tree_loss = np.load("../code/baseline/cnn-tree/1/0/0.01/20000/0.0/test-error-['dw', 'dr', 'hvac', 'fridge', 'mw'].npy").item()
order = ['dw', 'dr', 'hvac', 'fridge', 'mw']
cnn_tree_pred = {}
for idx, appliance in enumerate(order):
    print(idx, appliance)
    cnn_tree_pred[appliance] = cnn_pred[20000][idx].reshape(-1, 1, 112,24)

0 dw
1 dr
2 hvac
3 fridge
4 mw


In [152]:
print(cnn_individual_error)
print(cnn_tree_loss[20000])
print(rnn_individual_error)
print(rnn_tree_error)

{'hvac': 186.65683, 'fridge': 29.265575, 'dr': 37.524254, 'dw': 11.02479, 'mw': 7.3939905}
{'dw': 11.517038054217622, 'dr': 38.741945219055715, 'hvac': 167.2713076146099, 'fridge': 35.701260723504845, 'mw': 7.2681973578917578}
{'hvac': 112.15536943674897, 'fridge': 26.364538273818635, 'dr': 51.332113758065901, 'dw': 11.13229631636338, 'mw': 6.813213514919009}
{'fridge': 32.386288271646919, 'hvac': 90.562519457287365, 'dw': 11.031979601126308, 'dr': 49.031312089803237, 'mw': 8.9075847805505788}


In [156]:
error = {}
error['Individual RNN'] = rnn_individual_error
error['Tree RNN'] = rnn_tree_error
error['Individual CNN'] = cnn_individual_error
error['Tree CNN'] = cnn_tree_loss[20000]

In [158]:
pd.DataFrame(error).T

,dr,dw,fridge,hvac,mw
Individual CNN,37.524254,11.024790,29.265575,186.656830,7.393991
Individual RNN,51.332114,11.132296,26.364538,112.155369,6.813214
Tree CNN,38.741945,11.517038,35.701261,167.271308,7.268197
Tree RNN,49.031312,11.031980,32.386288,90.562519,8.907585
